In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pandasticsearch import Select, DataFrame

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from IPython.display import Image
image="https://i0.wp.com/sisypheanhigh.com/foot/wp-content/uploads/2015/08/sisyphus-red.png"
#image="https://thumbnail.imgbin.com/1/4/18/imgbin-sisyphus-firefighting-fAsnwcmm8Egyb1SbCWRh8mQT8_t.jpg"
Image(url=image)


In [ ]:
hostname="http://noname.us.cray.com:30200"
client =  Elasticsearch(hostname, http_compress=True)

In [ ]:
cluster_health=client.cluster.health()

In [ ]:
client_info=client.info()

In [ ]:
resp = client.search(
    index="shasta-logs-*",
    body={
        "size":0,
        "query": {
            "bool": {
              "must": [{
                  "match_all": {}
                },
                {
                  "range": {
                    "timereported": {
                      "gte": 1563847200000,
                      "lte": 1563978841842,
                      "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      },
        "_source":{
            "excludes":[]
        },
        "aggs": {
        "2": {
          "date_histogram": {
            "field": "timereported",
            "interval": "30s",
            "time_zone": "America/Chicago",
            "min_doc_count": 1
          }
        }
      }
    }
)

In [ ]:
#print(resp)
resp['hits']['total']

In [ ]:
table=resp['aggregations']['2']['buckets']

In [ ]:
pd_table=pd.DataFrame.from_dict(table)
pd_table.drop(columns="key", inplace=True)

pd_table.head(100)

In [ ]:
pd_table.columns

In [ ]:
pd_table.shape

In [ ]:
pd_table.hist(column='doc_count', color='#86bf91',grid=False,bins=100)

In [ ]:
# Bokeh libraries
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

In [ ]:
output_notebook()  # Render inline in a Jupyter Notebook

In [ ]:
ds = ColumnDataSource(pd_table)
print(ds.data)
p = figure(width=800,height=400, x_axis_type='datetime')
p.line(source=ds, x='key_as_string', y='doc_count')
show(p)